<a href="https://colab.research.google.com/github/mohammadreza-mohammadi94/Deep-Learning-CNN-Projects/blob/master/Ensemble-ResNet-EffNet-MobileNet-Cassava-Dataset/Cassava_Ensemble_ResNet_EffNet_MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

cassava_leaf_disease_classification_path = kagglehub.competition_download('cassava-leaf-disease-classification')

print('Data source import complete.')


# Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

2025-12-16 13:11:48.160079: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765890708.331999      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765890708.379235      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

# Configuration & Setups

In [ ]:
# Dataset path
BASE_DIR = "/kaggle/input/cassava-leaf-disease-classification"
TRAIN_IMG_DIR = os.path.join(BASE_DIR, "train_images")
TRAIN_CSV = os.path.join(BASE_DIR, "train.csv")

# Hyperparameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10
NUM_CLASSES = 5
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
# Load CSV
df = pd.read_csv(TRAIN_CSV)
df['image_path'] = df['image_id'].apply(lambda x: os.path.join(TRAIN_IMG_DIR, x))

In [ ]:
# Stratified Split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

print(f"✅ Train samples: {len(train_df)}")
print(f"✅ Val samples: {len(val_df)}")

✅ Train samples: 17117
✅ Val samples: 4280


# Data Pipeline `tf.data`

In [ ]:
def load_and_preprocess(path, label):
    img = tf.io.read_file(path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    return img, label

def augment(img, label):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    img = tf.image.random_brightness(img, max_delta=0.2)
    img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
    # Random rotation (90 deg) is safe for leaves
    k = tf.random.uniform([], minval=0, maxval=4, dtype=tf.int32)
    img = tf.image.rot90(img, k)
    return img, label

def create_dataset(dataframe, is_train=False):
    paths = dataframe['image_path'].values
    labels = dataframe['label'].values

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(load_and_preprocess, num_parallel_calls=AUTOTUNE)
    if is_train:
        ds = ds.cache()
        ds = ds.shuffle(1000)
        ds = ds.map(augment, num_parallel_calls=AUTOTUNE)

    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(AUTOTUNE)
    return ds

# Build pipeline
train_ds = create_dataset(train_df, is_train=True)
val_ds = create_dataset(val_df, is_train=False)

I0000 00:00:1765890722.959272      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1765890722.959863      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


# Build Models

In [ ]:
def build_model(model_name):
    inputs = layers.Input(shape=(224, 224, 3))
    if model_name == "ResNet50":
        x = tf.keras.applications.resnet50.preprocess_input(inputs)
        base_model = tf.keras.applications.ResNet50(
            include_top=False,
            weights='imagenet',
            input_tensor=x
        )
    elif model_name == "EfficientNetB0":
        # EfficientNet expects [0, 255] inputs (no manual scaling needed usually, but using keras util is safe)
        # Note: EfficientNet usually has scaling built-in, so we pass raw inputs or standard scaling
        x = inputs
        base_model = tf.keras.applications.EfficientNetB0(
            include_top=False,
            weights="imagenet",
            input_tensor=x
        )
    elif model_name == "MobileNetV2":
        x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
        base_model = tf.keras.applications.MobileNetV2(
            include_top=False,
            weights="imagenet",
            input_tensor=x
        )

    # Freeze the base model (Optional: Unfreeze for fine-tuning later)
    base_model.trainable = False

    # Custom Head
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

    model = models.Model(inputs, outputs, name=model_name)
    return model

# Training

In [ ]:
def train_model_stages(model_name, warmup_epochs=5, finetune_epochs=12):
    print("\n" + "=" * 50)
    print(f"Processing Model: {model_name}")
    print("=" * 50)

    # 1. Build Model
    model = build_model(model_name)

    # --- PHASE 1: Warm-up ---
    # In build_model, we assumed base_model.trainable = False.
    # So only the classification head is trainable now.

    print(f"\nPhase 1: Warm-up (Head Only)")
    model.compile(optimizer=keras.optimizers.Adam(1e-3),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(train_ds, validation_data=val_ds, epochs=warmup_epochs, verbose=1)

    # --- PHASE 2: Fine-Tuning (Smart Unfreezing) ---
    print(f"\nPhase 2: Fine-Tuning (Last 50 Layers & BN Frozen)")

    # Unfreeze the whole model first
    model.trainable = True

    # Access the Base Model (Backbone)
    # Usually it's the layer after Input and Preprocessing.
    # We iterate to find the 'Functional' model inside.
    base_model = None
    for layer in model.layers:
        if isinstance(layer, keras.Model):
            base_model = layer
            break

    if base_model:
        # 1. Freeze all layers except the last 50
        for layer in base_model.layers[:-50]:
            layer.trainable = False

        # 2. CRITICAL: Force BatchNormalization layers to stay frozen
        # This prevents breaking the learned statistics from ImageNet
        for layer in base_model.layers:
            if isinstance(layer, layers.BatchNormalization):
                layer.trainable = False

        print(f"   --> backbone configured: Top 50 layers unfrozen, BN layers frozen.")

    # Recompile with Low Learning Rate
    model.compile(optimizer=keras.optimizers.Adam(1e-6), # 10x smaller LR
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Callbacks
    checkpoint = keras.callbacks.ModelCheckpoint(
        f"{model_name}_best.keras", save_best_only=True, monitor='val_accuracy', mode='max', verbose=0
    )
    early_stop = keras.callbacks.EarlyStopping(
        monitor='val_accuracy', patience=4, restore_best_weights=True, verbose=1
    )

    # Train
    model.fit(train_ds, validation_data=val_ds, epochs=finetune_epochs,
              callbacks=[checkpoint, early_stop], verbose=1)

    print(f"{model_name} Training Completed.")
    return

In [ ]:
models_list = ['ResNet50', 'EfficientNetB0', 'MobileNetV2']

for m_name in models_list:
    train_model_stages(m_name)


Processing Model: ResNet50

Phase 1: Warm-up (Head Only)
Epoch 1/5
535/535 ━━━━━━━━━━━━━━━━━━━━ 77s 126ms/step - accuracy: 0.6348 - loss: 1.0660 - val_accuracy: 0.7189 - val_loss: 0.7699
Epoch 2/5
535/535 ━━━━━━━━━━━━━━━━━━━━ 59s 111ms/step - accuracy: 0.7076 - loss: 0.8201 - val_accuracy: 0.7187 - val_loss: 0.7752
Epoch 3/5
535/535 ━━━━━━━━━━━━━━━━━━━━ 59s 110ms/step - accuracy: 0.7179 - loss: 0.7836 - val_accuracy: 0.7180 - val_loss: 0.7881
Epoch 4/5
535/535 ━━━━━━━━━━━━━━━━━━━━ 59s 110ms/step - accuracy: 0.7243 - loss: 0.7717 - val_accuracy: 0.7304 - val_loss: 0.7543
Epoch 5/5
535/535 ━━━━━━━━━━━━━━━━━━━━ 58s 109ms/step - accuracy: 0.7217 - loss: 0.7775 - val_accuracy: 0.7348 - val_loss: 0.7501

Phase 2: Fine-Tuning (Last 50 Layers & BN Frozen)
Epoch 1/12
535/535 ━━━━━━━━━━━━━━━━━━━━ 76s 126ms/step - accuracy: 0.7334 - loss: 0.7562 - val_accuracy: 0.7383 - val_loss: 0.7347
Epoch 2/12
535/535 ━━━━━━━━━━━━━━━━━━━━ 60s 112ms/step - accuracy: 0.7343 - loss: 0.7459 - val_accuracy: 0.741

/tmp/ipykernel_47/4187861907.py:21: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(



Phase 1: Warm-up (Head Only)
Epoch 1/5
535/535 ━━━━━━━━━━━━━━━━━━━━ 35s 50ms/step - accuracy: 0.6250 - loss: 1.0874 - val_accuracy: 0.7119 - val_loss: 0.7674
Epoch 2/5
535/535 ━━━━━━━━━━━━━━━━━━━━ 22s 40ms/step - accuracy: 0.6986 - loss: 0.8235 - val_accuracy: 0.7248 - val_loss: 0.7398
Epoch 3/5
535/535 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 0.7064 - loss: 0.7918 - val_accuracy: 0.7320 - val_loss: 0.7356
Epoch 4/5
535/535 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.7103 - loss: 0.7930 - val_accuracy: 0.7273 - val_loss: 0.7409
Epoch 5/5
535/535 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.7177 - loss: 0.7846 - val_accuracy: 0.7357 - val_loss: 0.7327

Phase 2: Fine-Tuning (Last 50 Layers & BN Frozen)
Epoch 1/12
535/535 ━━━━━━━━━━━━━━━━━━━━ 35s 53ms/step - accuracy: 0.7239 - loss: 0.7635 - val_accuracy: 0.7383 - val_loss: 0.7259
Epoch 2/12
535/535 ━━━━━━━━━━━━━━━━━━━━ 22s 40ms/step - accuracy: 0.7199 - loss: 0.7661 - val_accuracy: 0.7381 - val_loss: 0.7215
Epoch 3/12
535

# Soft Voting

In [ ]:
print("\n" + "="*50)
print("Calculating Ensemble Results")
print("="*50)

# Load best saved models
m_resnet = keras.models.load_model('ResNet50_best.keras')
m_effnet = keras.models.load_model('EfficientNetB0_best.keras')
m_mobile = keras.models.load_model('MobileNetV2_best.keras')

# Get ground truth (Concatenate all batches from val_ds)
y_true = np.concatenate([y for x, y in val_ds], axis=0)

# Make Predictions
print(">> Predicting ResNet50...")
p_resnet = m_resnet.predict(val_ds, verbose=1)

print(">> Predicting EfficientNetB0...")
p_effnet = m_effnet.predict(val_ds, verbose=1)

print(">> Predicting MobileNetV2...")
p_mobile = m_mobile.predict(val_ds, verbose=1)

# Individual Accuracy
acc_r = accuracy_score(y_true, np.argmax(p_resnet, axis=1))
acc_e = accuracy_score(y_true, np.argmax(p_effnet, axis=1))
acc_m = accuracy_score(y_true, np.argmax(p_mobile, axis=1))

print(f"\nIndividual Accuracies:")
print(f"   ResNet50:       {acc_r:.2%}")
print(f"   EfficientNetB0: {acc_e:.2%}")
print(f"   MobileNetV2:    {acc_m:.2%}")

# Ensemble (Average Probability)
ensemble_preds = (p_resnet + p_effnet + p_mobile) / 3.0
ensemble_labels = np.argmax(ensemble_preds, axis=1)
acc_ensemble = accuracy_score(y_true, ensemble_labels)

print(f"\nEnsemble Accuracy: {acc_ensemble:.2%}")

# Check improvement
best_single = max(acc_r, acc_e, acc_m)
print(f"Boost over best single model: +{acc_ensemble - best_single:.2%}")


Calculating Ensemble Results
>> Predicting ResNet50...
134/134 ━━━━━━━━━━━━━━━━━━━━ 18s 108ms/step
>> Predicting EfficientNetB0...
134/134 ━━━━━━━━━━━━━━━━━━━━ 17s 88ms/step
>> Predicting MobileNetV2...
134/134 ━━━━━━━━━━━━━━━━━━━━ 11s 65ms/step

Individual Accuracies:
   ResNet50:       74.25%
   EfficientNetB0: 75.61%
   MobileNetV2:    74.49%

Ensemble Accuracy: 76.45%
Boost over best single model: +0.84%
